# extracting accessibility related patterns from ravelry

%%
## bootstrap our search for accessible ravelry patterns

there are several accessibility categories that need to be searched.
they are captured in the `searches` mapping where the key is the name/index
for the search and the value defines the query parameters.

In [1]:
    __import__("dotenv").load_dotenv()
    auth = (os.environ["RAVELRY_USERNAME"], os.environ["RAVELRY_PASSWORD"])
    import requests_cache, platformdirs
    cache = platformdirs.user_cache_path("a11yhood") / "ravelry"
    search_cache = requests_cache.CachedSession(cache / "search_responses.json")
    patterns_cache = requests_cache.CachedSession(cache / "patterns_responses.json")
    searches: dict = {
        "adaptive": {"pa": "adaptive"},
        "medical device access": {"pa": "medical-device-access"},
        "medical device support": {"pa": "medical-device-accessory"},
        # "mobility aid support": {"add": "mobility-aid-support"},
        # "other": {"add": "other-add-accessibility"},
        "therapy aid/toy": {"pa": "therapy-aid"},
        "medical": {"pc": "medical"}
    }
    seed_urls = ("https://api.ravelry.com/patterns/search.json?" + Series(searches).apply(urllib.parse.urlencode))
    first_pages = seed_urls.apply(compose(do(print), search_cache.get), auth=auth)

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>


In [2]:
%%
## finding the missing paginated search information

from the `first_pages` were learn about possible results we missed in the `first_pattern.paginator` attribute in the payload.

finding the missing paginated search information 
 from the first_pages were learn about possible results we missed in the first_pattern.paginator attribute in the payload.

In [3]:
    first_patterns = first_pages.methodcaller("json").series()
    paginated = first_patterns.paginator.series()
    paginated = paginated[paginated.page_count.gt(1)].drop(columns="page")
    paginated = paginated.join(paginated.page_count.add(1).apply(compose(list, partial(range, 2))).explode().rename("page"))

In [4]:
    other_urls = seed_urls[paginated.index] + "&page=" + paginated.page.astype(str)

In [6]:
    other_urls.apply(compose(do(print), search_cache.get), auth=auth)

<CachedResponse [200]: created: 2025-02-25 10:33:27 PST, expires: N/A (fresh), size: 157.46 KiB, request: GET https://api.ravelry.com/patterns/search.json?pa=adaptive&page=2>
<CachedResponse [200]: created: 2025-02-25 10:33:27 PST, expires: N/A (fresh), size: 67.26 KiB, request: GET https://api.ravelry.com/patterns/search.json?pa=medical-device-accessory&page=2>
<CachedResponse [200]: created: 2025-02-25 10:33:28 PST, expires: N/A (fresh), size: 330.39 KiB, request: GET https://api.ravelry.com/patterns/search.json?pa=therapy-aid&page=2>
<CachedResponse [200]: created: 2025-02-25 10:33:29 PST, expires: N/A (fresh), size: 350.23 KiB, request: GET https://api.ravelry.com/patterns/search.json?page=2&pc=medical>
<CachedResponse [200]: created: 2025-02-25 10:33:29 PST, expires: N/A (fresh), size: 59.80 KiB, request: GET https://api.ravelry.com/patterns/search.json?page=3&pc=medical>


adaptive                  <CachedResponse [200]: created: 2025-02-25 10:...
medical device support    <CachedResponse [200]: created: 2025-02-25 10:...
therapy aid/toy           <CachedResponse [200]: created: 2025-02-25 10:...
medical                   <CachedResponse [200]: created: 2025-02-25 10:...
medical                   <CachedResponse [200]: created: 2025-02-25 10:...
dtype: object

In [7]:
    patterns = (
        pandas.concat([seed_urls, other_urls])
        .apply(search_cache.get)
        .apply(operator.methodcaller("json"))
        .apply(operator.itemgetter("patterns"))
        .explode()
        .apply(Series)
        .drop_duplicates("permalink")
    )
    patterns = patterns.set_index(("https://www.ravelry.com/patterns/library/" + patterns["permalink"]).rename("url"))

partition the patterns into requests of 20 patterns at a time

In [8]:
    pattern_chuck_size = 20
    pattern_urls = "https://api.ravelry.com/patterns.json?ids=" + (
        patterns.id.groupby(pandas.RangeIndex(len(patterns)) // pattern_chuck_size)
        .agg(list)
        .apply(compose("+".join, map(str)))
    )

In [9]:
    (
        pattern_urls.apply(compose(patterns_cache.get), auth=auth)
        .apply(operator.methodcaller("json"))
        .apply(operator.itemgetter("patterns"))
        .apply(compose(list, dict.values))
        .explode()
        .apply(Series)
    )

,comments_count,created_at,currency,difficulty_average,difficulty_count,downloadable,favorites_count,free,gauge,gauge_divisor,...,languages,packs,printings,yarn_weight,craft,pattern_categories,pattern_attributes,pattern_author,photos,pattern_type
0,2,2007/10/01 14:26:16 -0400,USD,3.666667,9.0,True,280,True,28.0,4.0,...,"[{'code': 'en', 'id': 1, 'name': 'English', 'p...","[{'id': 442678, 'primary_pack_id': None, 'proj...","[{'created_at': '2009/11/30 09:09:38 -0500', '...","{'crochet_gauge': '', 'id': 5, 'knit_gauge': '...","{'id': 2, 'name': 'Knitting', 'permalink': 'kn...","[{'id': 885, 'name': 'Mid-calf', 'permalink': ...","[{'id': 3, 'permalink': 'unisex'}, {'id': 25, ...","{'crochet_pattern_count': 0, 'favorites_count'...","[{'id': 116608078, 'sort_order': 1, 'user_id':...","{'clothing': True, 'id': 2, 'name': 'Socks', '..."
0,6,2008/03/28 14:40:29 -0400,None,2.500000,14.0,True,1056,True,8.0,1.0,...,"[{'code': 'en', 'id': 1, 'name': 'English', 'p...","[{'id': 1686809, 'primary_pack_id': None, 'pro...","[{'created_at': '2008/09/09 13:35:17 -0400', '...","{'crochet_gauge': '', 'id': 5, 'knit_gauge': '...","{'id': 2, 'name': 'Knitting', 'permalink': 'kn...","[{'id': 885, 'name': 'Mid-calf', 'permalink': ...","[{'id': 10, 'permalink': 'adult'}, {'id': 23, ...","{'crochet_pattern_count': 0, 'favorites_count'...","[{'id': 3715705, 'sort_order': 1, 'user_id': 1...","{'clothing': True, 'id': 2, 'name': 'Socks', '..."
0,23,2008/05/17 10:10:10 -0400,None,2.788991,109.0,True,1989,True,24.0,4.0,...,"[{'code': 'en', 'id': 1, 'name': 'English', 'p...","[{'id': 124848079, 'primary_pack_id': None, 'p...","[{'created_at': '2010/01/20 20:10:31 -0500', '...","{'crochet_gauge': None, 'id': 12, 'knit_gauge'...","{'id': 2, 'name': 'Knitting', 'permalink': 'kn...","[{'id': 365, 'name': 'Legwarmers', 'permalink'...","[{'id': 3, 'permalink': 'unisex'}, {'id': 64, ...","{'crochet_pattern_count': 1, 'favorites_count'...","[{'id': 13669012, 'sort_order': 1, 'user_id': ...","{'clothing': True, 'id': 2, 'name': 'Socks', '..."
0,23,2009/01/14 12:30:32 -0500,USD,1.801282,156.0,True,2781,True,14.0,4.0,...,"[{'code': 'en', 'id': 1, 'name': 'English', 'p...",[],"[{'created_at': '2017/09/23 04:44:04 -0400', '...","{'crochet_gauge': None, 'id': 12, 'knit_gauge'...","{'id': 1, 'name': 'Crochet', 'permalink': 'cro...","[{'id': 350, 'name': 'Shawl / Wrap', 'permalin...","[{'id': 3, 'permalink': 'unisex'}, {'id': 204,...","{'crochet_pattern_count': 3, 'favorites_count'...","[{'id': 64077605, 'sort_order': 1, 'user_id': ...","{'clothing': True, 'id': 10, 'name': 'Shawl/Wr..."
0,13,2009/04/04 14:42:17 -0400,USD,1.811881,101.0,False,1936,False,8.0,4.0,...,"[{'code': 'en', 'id': 1, 'name': 'English', 'p...","[{'id': 6186675, 'primary_pack_id': None, 'pro...","[{'created_at': '2009/08/01 16:54:19 -0400', '...","{'crochet_gauge': '', 'id': 4, 'knit_gauge': '...","{'id': 2, 'name': 'Knitting', 'permalink': 'kn...","[{'id': 350, 'name': 'Shawl / Wrap', 'permalin...","[{'id': 2, 'permalink': 'female'}, {'id': 10, ...","{'crochet_pattern_count': 6, 'favorites_count'...","[{'id': 23579035, 'sort_order': 1, 'user_id': ...","{'clothing': True, 'id': 10, 'name': 'Shawl/Wr..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32,1,2020/11/15 06:36:01 -0500,EUR,0.000000,NaN,False,73,False,NaN,4.0,...,"[{'code': 'en', 'id': 1, 'name': 'English', 'p...","[{'id': 129584545, 'primary_pack_id': None, 'p...","[{'created_at': '2020/11/15 06:36:01 -0500', '...",NaN,"{'id': 2, 'name': 'Knitting', 'permalink': 'kn...","[{'id': 440, 'name': 'Medical', 'permalink': '...","[{'id': 327, 'permalink': 'other-accessibility'}]","{'crochet_pattern_count': 5, 'favorites_count'...",[],"{'clothing': False, 'id': 21, 'name': 'Home', ..."
32,0,2020/11/16 04:39:44 -0500,EUR,0.000000,NaN,False,11,False,NaN,4.0,...,"[{'code': 'en', 'id': 1, 'name': 'English', 'p...","[{'id': 129584549, 'primary_pack_id': None, 'p...","[{'created_at': '2020/11/16 04:39:45 -0500', '..."